In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [ ]:
def load_utk_dataset(dataset_path):
    images = []
    ages = []
    for file in os.listdir(dataset_path):
        if file.endswith(".jpg") or file.endswith(".png"):
            try:
                age = int(file.split('_')[0])  # Extract the age from the filename
                img_path = os.path.join(dataset_path, file)
                img = cv2.imread(img_path)
                if img is not None:  # Ensure the image was loaded correctly
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (200, 200))
                    images.append(img)
                    ages.append(age)
                else:
                    print(f"Warning: Failed to load image {img_path}")
            except Exception as e:
                print(f"Warning: Skipping file {file} due to error: {e}")
    return np.array(images), np.array(ages)

dataset_path = '/kaggle/input/utkface-new/crop_part1'  # Correct path to the dataset
images, ages = load_utk_dataset(dataset_path)

In [ ]:

images = images / 255.0


X_train, X_test, y_train, y_test = train_test_split(images, ages, test_size=0.2, random_state=42)

In [ ]:
def create_cnn_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear'))  # Linear activation for regression

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

model = create_cnn_model()
model.summary()


In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

# Fit the model
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_test, y_test),
                    epochs=100,
                    steps_per_epoch=len(X_train) // 32,
                    verbose=1)


In [ ]:
# Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae}")

# Plot training history
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.plot(history.history['mae'], label='train_mae')
plt.plot(history.history['val_mae'], label='val_mae')
plt.xlabel('Epochs')
plt.ylabel('Loss/MAE')
plt.legend()
plt.show()


In [ ]:
model.save('age_prediction_modelv2.h5')


In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

In [ ]:
model = load_model('/kaggle/working/age_prediction_model.h5')

In [5]:
def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (200, 200))
    img = img / 255.0  # Normalize the image
    return img
def predict_age(img_path):
    img = preprocess_image(img_path)
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    predicted_age = model.predict(img)
    return predicted_age[0][0]



In [ ]:
image_path = '/kaggle/input/actress/images (1).jpg'
predicted_age = predict_age(image_path)
print(f"Predicted Age: {predicted_age}")

In [ ]:
def calculate_regression_accuracy(y_true, y_pred, tolerance=5):
    """
    Calculate the accuracy for regression by checking how many predictions fall within the tolerance of actual values.

    Parameters:
    y_true (numpy array): Actual values
    y_pred (numpy array): Predicted values
    tolerance (int): The maximum difference allowed for a prediction to be considered correct

    Returns:
    float: The accuracy percentage
    """
    accurate_predictions = np.abs(y_true - y_pred) <= tolerance
    accuracy = np.mean(accurate_predictions) * 100
    return accuracy

#Make predictions with the best model
y_pred = model.predict(X_test)

#Calculate the accuracy within a tolerance of 5 years
tolerance = 5
regression_accuracy = calculate_regression_accuracy(y_test, y_pred, tolerance)
print(f"Regression Accuracy within {tolerance} years: {regression_accuracy:.2f}%")

In [ ]:
#accuracy
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae}")